# Ejercicio 4

**(a) Lea los archivos de datos y cárguelos en dos dataframe o matrices X, y. En el caso de X es extremadamente importante que mantenga el formato disperso (sparse) (¿porqué?).**

In [1]:
import os.path

data_base_path = "./data"

In [2]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from scipy.io import mmread

X_train = csr_matrix(mmread(os.path.join(data_base_path, 'train.x.mm')))
y_train = np.loadtxt(os.path.join(data_base_path, 'train.y.dat'))

X_dev = csr_matrix(mmread(os.path.join(data_base_path, 'dev.x.mm')))
y_dev = np.loadtxt(os.path.join(data_base_path, 'dev.y.dat'))

X_test = csr_matrix(mmread(os.path.join(data_base_path, 'test.x.mm')))
y_test = np.loadtxt(os.path.join(data_base_path, 'test.y.dat'))

Se usa una matriz sparse ya que, producto de que hay muchos datos vacios (normal en texto). Así se comprime la matriz y queda en un tamaño manejable.

**(b) Construya un modelo lineal que obtenga un coeficiente de determinación (sobre el conjunto de pruebas) de al menos 0.75.**

In [3]:
X_train.get_shape()

(1147, 145256)

Se observa que hay 1147 peliculas cada una con 145256 atributos.

In [4]:
#from sklearn.preprocessing import StandardScaler

#x_scaler = StandardScaler(with_mean=False)

#X_train = x_scaler.fit_transform(X_train, y_train)
#X_test = x_scaler.transform(X_test, y_test)

Se prueba escalar los datos pero esto devuelve peores resultados.

In [5]:
from sklearn.feature_selection import SelectKBest, chi2

ch2 = SelectKBest(chi2, k=29000)

X_train = ch2.fit_transform(X_train, y_train)
X_dev = ch2.transform(X_dev)
X_test = ch2.transform(X_test)

Se utiliza la extracción de caracteristicas KBest mediante la metrica chi cuadrado. Esto ya que documentación de sklearn muestra que es utilizada cuando la data esta basada en texto (como es el caso).

La cantidad de catacteristicas se obtiene mediante prueba y error.

Fuente: http://scikit-learn.org/stable/auto_examples/text/document_classification_20newsgroups.html#example-text-document-classification-20newsgroups-py

In [9]:
import sklearn.linear_model as lm

model = lm.LinearRegression(fit_intercept = False)
model.fit(X_train, y_train)

print "R2=%f" % model.score(X_test, y_test)

R2=0.600107


Con una regresión linear ordinaria se logra un R² de 0.6.

In [7]:
import sklearn.linear_model as lm

model = lm.LassoCV(n_jobs=7)

model.fit(X_train, y_train)

print "R2=%f" % model.score(X_test, y_test)

R2=0.504787


Se prueba además con Lasso pero se obtiene peor resultados. Normalizar los datos no cambia en nada el R².

Se utiliza la clase LassoCV que recorre varios valores de alpha y elije el mejor en el conjunto de entrenamiento.

El alpha de Lasso es:

In [8]:
model.alpha_

15405802.603399234

In [12]:
import sklearn.linear_model as lm

model = lm.RidgeCV()

model.fit(X_train, y_train)

print "R2=%f" % model.score(X_test, y_test)

R2=0.563866


Tampoco tenemos suerte con Ridge.

El alpha de Ridge es:

In [13]:
model.alpha_

10.0

Se prueba con ElasticNet que mezcla ambos modelos:

In [ ]:
import sklearn.linear_model as lm

model = lm.ElasticNetCV(l1_ratio=np.arange(0, 1, 0.1), alphas=np.arange(0, 1, 0.1), n_jobs=7, verbose=1)

model.fit(X_train, y_train)

print "R2=%f" % model.score(X_test, y_test)

Y luego se ejecuta en un rango más acotado conforme a los optimos obtenidos anteriormente:

In [6]:
import sklearn.linear_model as lm

model = lm.ElasticNetCV(l1_ratio=np.arange(0, 0.1, 0.01), alphas=np.arange(0.8, 1, 0.01), n_jobs=7, verbose=1)

model.fit(X_train, y_train)

print "R2=%f" % model.score(X_test, y_test)

/usr/local/lib/python2.7/dist-packages/sklearn/linear_model/coordinate_descent.py:466: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations
  ConvergenceWarning)
........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................[Parallel(n_jobs=7)]: Done  30 out of  30 | elapsed: 1101.0min finished


R2=0.608121


Se obtienen el mejor resultado hasta el momento.

A continuación en vez de realizar cross-validation se usa el conjunto de validación ("dev") y modificando los parametros manualmente:

In [6]:
import sklearn.linear_model as lm

model = lm.ElasticNet(alpha = 0.05, l1_ratio=0.775)
model.fit(X_train, y_train)

print "R2=%f" % model.score(X_dev, y_dev)

R2=0.605732


/usr/local/lib/python2.7/dist-packages/sklearn/linear_model/coordinate_descent.py:466: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations
  ConvergenceWarning)


Dando:

In [7]:
print "R2=%f" % model.score(X_test, y_test)

R2=0.634180


Cabe mencionar que se probo con tecnicas como Grid Search y Randomized Parameter Optimization pero sus tiempos de computo fueron excesivos.